In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install streamlit

In [3]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
from torch import nn
import torch.nn.functional as F
import streamlit as st

In [22]:
pip uninstall torch -y

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124


In [23]:
pip cache purge  # optional: cleans old wheels

Files removed: 78


In [24]:
pip install torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.9/184.9 MB 6.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.6 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.5 MB/s eta 0:00:00
     ━━━

In [27]:
pip install torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [14]:
pip install streamlit

In [5]:
model_path = '/content/drive/MyDrive/Models/Em_model_22_1.5e-06lr.pth'

In [4]:
from transformers import AutoTokenizer, AutoModel
from torch import nn
import torch.nn.functional as F

# Attributes
attributes = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']

# Model definition
class Emotion_Sentence_Classifier(nn.Module):
    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        self.pretrained_model = AutoModel.from_pretrained(config['model_name'], return_dict=True)
        self.hidden = nn.Linear(self.pretrained_model.config.hidden_size, 64)
        self.classifier = nn.Linear(64, self.config['n_labels'])
        nn.init.xavier_uniform_(self.classifier.weight)
        self.loss_func = nn.BCEWithLogitsLoss(reduction='mean')
        self.dropout = nn.Dropout()

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = torch.mean(output.last_hidden_state, 1)
        pooled_output = self.dropout(pooled_output)
        pooled_output = self.hidden(pooled_output)
        pooled_output = F.relu(pooled_output)
        logits = self.classifier(pooled_output)
        loss = 0
        if labels is not None:
            loss = self.loss_func(logits.view(-1, self.config['n_labels']), labels.view(-1, self.config['n_labels']))
        return loss, logits

# Load config and model
config = {
    'model_name': 'distilroberta-base',
    'n_labels': len(attributes),
    'batch_size': 128,
    'lr': 1.5e-6,
    'n_epochs': 10
}

# Initialize model
model = Emotion_Sentence_Classifier(config)

# Load model state (change path if needed)
model_state_dict = torch.load('/content/Em_model_22_1.5e-06lr.pth', map_location=torch.device('cpu'))
model.load_state_dict(model_state_dict)
model.eval()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')

# Inference function
def inference(sentence):
    encoded = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        _, output = model(encoded['input_ids'], encoded['attention_mask'], None)
        prediction = (torch.sigmoid(output) > 0.5).int()
        em = [attributes[i] for i, val in enumerate(prediction[0]) if val == 1]
    return em

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
while True:
    sentence = input("Enter a sentence (or type 'exit' to quit): ")
    if sentence.lower() == 'exit':
        print("Goodbye!")
        break
    emotions = inference(sentence)
    if emotions:
        print("Detected Emotions:", ", ".join(emotions))
    else:
        print("No strong emotions detected.")


Enter a sentence (or type 'exit' to quit): They threw me a party I wasn’t expecting — I was completely stunned!
No strong emotions detected.
Enter a sentence (or type 'exit' to quit): I know I can count on you no matter what happens.
Detected Emotions: sadness
